# **GPAW Tutorial for DeepChem**

Author : Rakshit Kr. Singh : [Website](https://sudo-rsingh.github.io/) : [LinkedIn](https://www.linkedin.com/in/rakshit-singh-ai/) : [GitHub](https://github.com/sudo-rsingh)

## **Introduction**

GPAW (Grid-Based Projector-Augmented Wave Method) is a Python-based software for performing density functional theory (DFT) calculations. DFT is widely used in computational materials science to calculate electronic structure, total energy, forces, and other properties of materials at the atomic scale. This tutorial guides you through performing unit cell calculations with GPAW and explains all relevant concepts.

### **Understanding Density Functional Theory (DFT)**

Density Functional Theory (DFT) is a quantum mechanical modeling method used in computational chemistry and physics to investigate the electronic structure of many-body systems, particularly atoms, molecules, and solids. It is a crucial tool for understanding and predicting the properties of materials, from the behavior of individual atoms to the macroscopic properties of solid-state materials.

The central idea of DFT is that the properties of a many-electron system can be determined by knowing the electron density, rather than the many-body wavefunction. In other words, instead of calculating the full wavefunction of a system (which is a complex and computationally expensive task), DFT simplifies the problem by focusing on the electron density, which is a function of just three spatial coordinates, not all the individual positions of the electrons.

#### **DFT is grounded in two theorems proposed by Hohenberg and Kohn in 1964**:

- **First Theorem**: The ground-state properties of a many-electron system are uniquely determined by the electron density.
- **Second Theorem**: The electron density that minimizes the total energy of the system is the true ground-state density.

While the Hohenberg-Kohn theorems show the existence of a functional for the total energy, the exact form of this functional is not known. However, Kohn and Sham developed a practical approach in 1965 that approximates the ground-state electron density and energy using a set of non-interacting reference systems. The solution involves solving a set of Kohn-Sham equations that describe the behavior of non-interacting electrons under an effective potential, which includes the effects of electron-electron interactions.

In physics and quantum chemistry, specifically density functional theory, the Kohn–Sham equation is the non-interacting Schrödinger equation (more clearly, Schrödinger-like equation) of a fictitious system (the "Kohn–Sham system") of non-interacting particles (typically electrons) that generate the same density as any given system of interacting particles.

**This eigenvalue equation is the typical representation of the Kohn–Sham equations**:

$$ \left[ -\frac{\hbar^2}{2m} \nabla^2 + V_\text{eff}(\mathbf{r}) \right] \psi_i(\mathbf{r}) = \epsilon_i \psi_i(\mathbf{r}) $$

where $ V_{eff}(r) $ is the effective potential that includes both the external potential and the electron-electron interaction effects.


## **Setup**

To install and set up GPAW, you need to ensure that all its dependencies, including libraries for numerical computations and compilers, are properly installed. The steps below outline a typical installation process:

1. **Install Required Libraries**: Use conda to install essential libraries like libxc and blas, which are crucial for GPAW's computational routines.
2. **System Updates and Compiler Installation**: Update the system's package list and install the GCC compiler, as it is required for compiling GPAW.
3. **Install GPAW**: Install GPAW directly from its GitLab repository using pip. Alternatively, you can use pip install gpaw for the latest stable version.
4. **Installation of PAW datasets**: After installation, run the following command to download necessary auxiliary data for GPAW.

In [ ]:
%conda install conda-forge::libxc
%conda install -c conda-forge blas
%apt update
%apt install gcc
%pip install git+https://gitlab.com/gpaw/gpaw.git  #pip install gpaw
%gpaw install-data .

## **Setting up Computational Environment (ASE) and defining Unit Cell Structure**

The **Atomic Simulation Environment (ASE)** is a Python library for setting up, running, and analyzing atomistic simulations across various computational tools, including DFT codes like GPAW, VASP, and Quantum ESPRESSO. It allows users to create and manipulate atomic structures, perform geometry optimizations, molecular dynamics simulations, and compute properties like energies, forces, and vibrational spectra. ASE's core revolves around the **Atoms** object, which represents atomic systems, and **Calculators**, which interface with external simulation engines. It supports building crystals, surfaces, and clusters while enabling custom workflows through Python scripting. With its flexibility, extensibility, and integration of visualization and analysis tools, ASE is widely used for material science, catalysis, and chemical reaction studies.

A unit cell is the smallest repeating structural unit of a crystal lattice that represents the overall symmetry and structure of the crystal. By repeating the unit cell in three-dimensional space, the entire crystal lattice can be reconstructed. Understanding unit cells is fundamental in crystallography, materials science, and solid-state physics.

**Diamond Structure**: Silicon atoms form a tetrahedral network with a face-centered cubic (FCC) lattice, characteristic of semiconductors like Si and Ge.

Structures could be one of sc, fcc, bcc, tetragonal, bct, hcp, rhombohedral, orthorhombic, mcl, diamond, zincblende, rocksalt, cesiumchloride, fluorite or wurtzite.

In [19]:
from ase.build import bulk


# Create silicon crystal
si = bulk('Si', 'diamond', a=5.43)

### **Custom Unit Cell**

To define a unit cell manually, specify its lattice vectors and atomic positions:

- **Lattice Vectors**: These define the periodicity of the unit cell. The example shows a cubic cell with equal edge lengths.
- **Fractional Coordinates**: These are normalized to the cell dimensions, making the definition scalable.
- **Periodic Boundary Conditions (PBC)**: Essential for simulating bulk properties by eliminating surface effects.

In [20]:
from ase import Atoms


# Define lattice vectors
a = 5.43  # Lattice constant
cell = [(a, 0, 0),
        (0, a, 0),
        (0, 0, a)]

# Define atomic positions (fractional coordinates)
positions = [(0, 0, 0),
             (0.25, 0.25, 0.25)]

# Create structure
atoms = Atoms('Si2',
              scaled_positions=positions,
              cell=cell,
              pbc=True)  # Periodic boundary conditions

## **Setting up GPAW Calculator for DFT Calculations**

GPAW (Grid-based Projector Augmented Wave) is an open-source Python library for performing density functional theory (DFT) calculations. It uses real-space grids and the projector augmented wave (PAW) method to achieve high accuracy in electronic structure calculations for molecules, surfaces, and bulk materials. GPAW supports various computational modes, including real-space grids, plane waves, and linear combinations of atomic orbitals (LCAO), making it versatile for different system sizes and properties. It integrates seamlessly with the Atomic Simulation Environment (ASE) for structure creation, manipulation, and simulation workflows. GPAW can calculate electronic properties such as band structures, density of states, and optical spectra, and it supports parallelization for large-scale simulations. Its flexibility, combined with Python-based scripting, makes it a powerful tool for computational material science and chemistry research.

- **Exchange-Correlation Functional (PBE):** The Perdew-Burke-Ernzerhof (PBE) functional is a widely used approximation in DFT that balances accuracy and computational efficiency for various systems.
- **k-Point Mesh:** The k-points discretize the Brillouin zone, ensuring accurate integration of electronic states. A denser mesh improves accuracy but increases computational cost.


In [21]:
from gpaw import GPAW


# Define calculator parameters
calc = GPAW(mode='pw',        # Plane-wave mode
            xc='PBE',         # Exchange-correlation functional
            kpts=(4, 4, 4),   # k-point mesh
            txt='output.txt') # Output file

# Adding calculator to the ase environment
si.calc = calc

## **Ground State Energy calculation**

The ground state energy represents the minimum energy configuration of a system, obtained by solving the Kohn-Sham equations within Density Functional Theory (DFT). In GPAW, this is achieved by defining a structure and setting up a calculator with appropriate parameters such as exchange-correlation functional, k-point sampling, and energy cutoff.

In [22]:
# Get total energy
si.get_potential_energy()

-10.788084128067034

## **Cell Manipulation**

### **Changing Cell Parameters**

You can scale or modify the lattice vectors to simulate thermal expansion or structural distortions:

- **Scaling**: Uniformly increases the cell size, useful for studying volumetric strain or thermal expansion.
- **Directional** Modification: Alters a specific dimension, enabling the simulation of uniaxial stress or strain.

In [23]:
# Scale the cell
atoms.set_cell(atoms.get_cell() * 1.02, scale_atoms=True)

# Change a specific lattice vector
cell = atoms.get_cell()
cell[0] *= 1.1  # Stretch along x-axis
atoms.set_cell(cell, scale_atoms=True)

### **Strain Applications**

Strain engineering modifies material properties by adjusting lattice parameters:

- Strain tensors describe the deformation of a unit cell. Biaxial strain alters two dimensions while maintaining the third, common in thin-film studies.

In [26]:
import numpy as np


def apply_strain(atoms, strain_tensor):
    """Apply strain to unit cell"""
    cell = atoms.get_cell()
    strained_cell = np.dot(cell, strain_tensor)
    atoms.set_cell(strained_cell, scale_atoms=True)

# Example: Apply 2% biaxial strain
strain = np.eye(3)
strain[0,0] = strain[1,1] = 1.02
apply_strain(atoms, strain)

atoms

Atoms(symbols='Si2', pbc=True, cell=[6.2143092, 5.649372, 5.5386])

## **Cell Optimization**

Cell optimization is the process of adjusting the atomic positions and lattice parameters of a system to find the equilibrium structure with the lowest total energy. This is essential for accurately predicting material properties, as it ensures the simulated structure reflects its most stable configuration under given conditions. Cell optimization involves minimizing forces on atoms and the stress on the unit cell.

In GPAW, cell optimization can be performed using optimizers from the Atomic Simulation Environment (ASE), such as BFGS or LBFGS, which iteratively adjust the cell and atomic positions.

During optimization, forces acting on atoms are reduced below a specified threshold (e.g., 0.01 eV/Å), and the stress tensor is minimized to ensure mechanical equilibrium of the cell. This process is critical for studying materials under strain, phase transitions, and mechanical properties.

In [32]:
from gpaw import GPAW
from ase.build import bulk
from ase.optimize import BFGS
from ase.filters import UnitCellFilter

# Create a silicon crystal
si = bulk('Si', 'diamond', a=5.43)

# Define the calculator
calc = GPAW(mode='pw',        # Plane-wave mode
            xc='PBE',         # Exchange-correlation functional
            kpts=(4, 4, 4),   # k-point mesh
            txt="output.txt")
si.calc = calc

# Apply a unit cell filter to allow both atomic and cell optimization
ucf = UnitCellFilter(si)

# Run the optimization
opt = BFGS(ucf, trajectory='cell_optimization.traj')
opt.run(fmax=0.0001)  # Convergence criterion for forces (eV/Å)

# Get the optimized cell and energy
optimized_cell = si.get_cell()
optimized_energy = si.get_potential_energy()
print(f"Optimized cell: {optimized_cell}")
print(f"Optimized energy: {optimized_energy:.6f} eV")

      Step     Time          Energy          fmax
BFGS:    0 20:54:32      -10.788377        0.291028
BFGS:    1 20:54:33      -10.791563        0.216154
BFGS:    2 20:54:35      -10.795613        0.008368
BFGS:    3 20:54:37      -10.795619        0.000254
BFGS:    4 20:54:39      -10.795619        0.000000
Optimized cell: Cell([[-1.064167253964874e-18, 2.737613984722096, 2.737613984722096], [2.737613984722096, 1.723637505385204e-18, 2.737613984722096], [2.737613984722096, 2.737613984722096, 7.493170327501914e-19]])
Optimized energy: -10.795367 eV


## **Symmetry Considerations**

Symmetry plays a crucial role in computational materials science, significantly reducing the computational cost and improving the accuracy of simulations. By exploiting the inherent symmetry of a crystal structure, the number of atoms in the unit cell and the k-points in the Brillouin zone can be minimized while preserving the physical properties of the system. Symmetry also ensures that the results are consistent with the physical constraints of the material.

In simulations, symmetries are often described using **space groups**, which encode the symmetry operations of a crystal, such as rotations, reflections, and translations. GPAW, in conjunction with ASE, can leverage symmetry to construct crystal structures efficiently.

### **Benefits of Symmetry Considerations:**
- **Reduced Computational Cost:** Fewer atoms and k-points need to be calculated due to symmetry constraints.
- **Enhanced Accuracy:** Symmetry ensures that the computed properties are consistent with the physical laws governing the system.
- **Simplified Analysis:** Results such as electronic band structures are easier to interpret when symmetry is preserved.

### **Applications of Symmetry in DFT:**
- **Electronic Band Structure:** Symmetry simplifies the calculation of electronic states and reduces the k-point sampling needed.
- **Mechanical Properties:** Symmetry constraints help in studying anisotropic properties like elasticity and piezoelectricity.
- **Phase Transitions:** Symmetry breaking or enhancement can be studied to predict phase changes.

Understanding and incorporating symmetry is essential for efficient and accurate simulations in computational material science.

In [34]:
from ase.spacegroup import crystal
from gpaw import GPAW

# Create a silicon crystal with symmetry
si = crystal('Si',  # Element
             basis=[(0, 0, 0)],  # Atomic positions in fractional coordinates
             spacegroup=227,     # Fd-3m space group for diamond structure
             cellpar=[5.43, 5.43, 5.43, 90, 90, 90])  # Cell parameters

# Define the calculator
calc = GPAW(mode='pw', xc='PBE', kpts=(4, 4, 4), txt="output.txt")
si.calc = calc

# Calculate total energy
energy = si.get_potential_energy()
print(f"Total energy with symmetry consideration: {energy:.6f} eV")

Total energy with symmetry consideration: -43.152283 eV


## **Citing This Tutorial**

If you found this tutorial useful please consider citing it using the provided BibTeX.

```bibtex
@manual{
 title={GPAW Unit Cell Tutorial},
 organization={DeepChem},
 author={Singh, Rakshit kr.},
 howpublished = {\url{https://github.com/deepchem/deepchem/blob/master/examples/tutorials/ODE_Solving.ipynb}}, 
 year={2025}, 
} 
```

## **Congratulations! Time to join the Community!**

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:


### **Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)**
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.


### **Join the DeepChem Discord**
The DeepChem [Discord](https://discord.gg/SxSzjRRDMA) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!